# Problem 1
a. Build a Convolutional Neural Network, like what we built in lectures (without skip connections), to classify the images across all 10 classes in CIFAR 10. You need to adjust the fully connected layer at the end properly with respect to the number of output classes. Train your network for 300 epochs. Report your training time, training loss, and evaluation accuracy after 300 epochs. Analyze your results in your report and compare them against a fully connected network (homework 2) on training time, achieved accuracy, and model size. Make sure to submit your code by providing the GitHub URL of your course repository for this course.

In [ ]:
# Import Libraries
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
data_path = '../data-unversioned/p1ch7/'
from torchvision import transforms

device= (torch.device('cuda') if torch.cuda.is_available()
         else torch.device('cpu'))
print(f"Training on device {device}.")

# Preprocess images
preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )])

# Store validation and training data, preprocess data
cifar10 = datasets.CIFAR10(data_path, train=True, download=True, transform = preprocess)
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True, transform = preprocess)

train_loader = torch.utils.data.DataLoader(cifar10, batch_size=256, shuffle = True)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=256, shuffle = False)

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size = 3, padding = 1)
        self.conv2 = nn.Conv2d(32, 16, kernel_size = 3, padding = 1)
        self.fc1 = nn.Linear(8 * 8 * 16, 32)
        self.fc2 = nn.Linear(32, 10)
        
    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = F.max_pool2d(torch.relu(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * 16)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out



In [91]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader, val_loader):
# Iterates through each epoch
    model.train()
    for epoch in range(1, n_epochs+1):
        loss_train = 0.0
        for imgs, labels in train_loader:

    # Calculate training loss
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            train_loss = loss_fn(outputs, labels)
    
    # Zeros gradient for backpropagation and gradient calculation
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()
    
    # Calculate validation loss and accuracy
        correct = 0
        count = 0
        
    # Put model in eval mode
        model.eval()
        
    # Disables gradients for validation
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                batch_size = imgs.shape[0]
                outputs = model(imgs)
                _, label_p = torch.max(outputs, dim=1)
                count += labels.shape[0]
                correct += int((label_p == labels).sum())
                val_loss = loss_fn(outputs, labels)
    
    # Calculate accuracy
        acc = correct/count
        
        if epoch == 1 or epoch % 20 == 0:
            print('{} Epoch: {}, Training Loss: {}, Validation Loss {}, Accuracy {}'.format(
                datetime.datetime.now(), epoch, float(train_loss), float(val_loss), acc))
    

In [3]:
learning_rate = 1e-2
model = Net()
model = model.to(device=device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
n_epochs = 300

training_loop(n_epochs, optimizer, model, loss_fn, train_loader, val_loader)

2022-03-27 22:09:54.926980 Epoch: 1, Training Loss: 2.13584041595459, Validation Loss 2.1027963161468506, Accuracy 0.2252
2022-03-27 22:13:09.765961 Epoch: 20, Training Loss: 1.3218696117401123, Validation Loss 1.157007098197937, Accuracy 0.5746
2022-03-27 22:16:34.271728 Epoch: 40, Training Loss: 0.8693691492080688, Validation Loss 0.9199755787849426, Accuracy 0.6507
2022-03-27 22:20:01.889661 Epoch: 60, Training Loss: 0.7215105891227722, Validation Loss 0.7573404312133789, Accuracy 0.6568
2022-03-27 22:23:31.719490 Epoch: 80, Training Loss: 0.6601418256759644, Validation Loss 0.7964374423027039, Accuracy 0.6914
2022-03-27 22:27:00.277229 Epoch: 100, Training Loss: 0.8483656048774719, Validation Loss 0.9034740924835205, Accuracy 0.6782
2022-03-27 22:30:30.463864 Epoch: 120, Training Loss: 0.7288081049919128, Validation Loss 0.9072661995887756, Accuracy 0.6718
2022-03-27 22:33:58.552701 Epoch: 140, Training Loss: 0.4333290159702301, Validation Loss 0.6648874282836914, Accuracy 0.6762
2

b. Extend your CNN by adding one more additional convolution layer followed by an activation function and pooling function. You also need to adjust your fully connected layer properly with respect to intermediate feature dimensions. Train your network for 300 epochs. Report your training time, loss, and evaluation accuracy after 300 epochs. Analyze your results in your report and compare your model size and accuracy over the baseline implementation in Problem1.a. Do you see any over-fitting? Make sure to submit your code by providing the GitHub URL of your course repository for this course.

In [92]:
class ExtendedNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size = 3, padding = 1)
        self.conv2 = nn.Conv2d(32, 16, kernel_size = 3, padding = 1)
        self.conv3 = nn.Conv2d(16, 8, kernel_size = 3, padding = 1)
        self.fc1 = nn.Linear(4 * 4 * 8, 32)
        self.fc2 = nn.Linear(32, 10)
        
    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = F.max_pool2d(torch.relu(self.conv2(out)), 2)
        out = F.max_pool2d(torch.relu(self.conv3(out)), 2)
        out = out.view(-1, 4 * 4 * 8)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [93]:
learning_rate = 1e-2
model = ExtendedNet()
model = model.to(device=device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
n_epochs = 300

training_loop(n_epochs, optimizer, model, loss_fn, train_loader, val_loader)

2022-03-29 03:09:31.192759 Epoch: 1, Training Loss: 2.2898147106170654, Validation Loss 2.2693073749542236, Accuracy 0.1128
2022-03-29 03:12:50.209251 Epoch: 20, Training Loss: 1.4860203266143799, Validation Loss 1.5341228246688843, Accuracy 0.487
2022-03-29 03:16:18.182437 Epoch: 40, Training Loss: 1.1962817907333374, Validation Loss 1.1573352813720703, Accuracy 0.5747
2022-03-29 03:19:45.687222 Epoch: 60, Training Loss: 1.1376391649246216, Validation Loss 0.998846173286438, Accuracy 0.5996
2022-03-29 03:23:13.445723 Epoch: 80, Training Loss: 0.9487059712409973, Validation Loss 1.010607123374939, Accuracy 0.6537
2022-03-29 03:26:41.489972 Epoch: 100, Training Loss: 0.7791354060173035, Validation Loss 0.7740625739097595, Accuracy 0.6643
2022-03-29 03:30:09.737894 Epoch: 120, Training Loss: 1.0598613023757935, Validation Loss 0.9514034390449524, Accuracy 0.6305
2022-03-29 03:33:37.688061 Epoch: 140, Training Loss: 0.5545362234115601, Validation Loss 0.8162549734115601, Accuracy 0.6724
2

# Problem 2
a. Build a ResNet based Convolutional Neural Network, like what we built in lectures (with skip connections), to classify the images across all 10 classes in CIFAR 10. For this problem, let’s use 10 blocks for ResNet and call it ResNet-10. Use the similar dimensions and channels as we need in lectures. Train your network for 300 epochs. Report your training time, training loss, and evaluation accuracy after 300 epochs. Analyze your results in your report and compare them against problem 1.b on training time, achieved accuracy, and model size. Make sure to submit your code by providing the GitHub URL of your course repository for this course.

In [49]:
class ResBlock(nn.Module):
    def __init__(self, n_chans):
        super(ResBlock, self).__init__()
        self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3, padding=1, bias=True)
        
    def forward(self, x):
        out = self.conv(x)
        out = torch.relu(out)
        return out + x

In [50]:
class ResNetDeep(nn.Module):
    def __init__(self, n_chans1=32, n_blocks=10):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1, bias=True)
        self.resblocks = nn.Sequential(
            *(n_blocks * [ResBlock(n_chans = n_chans1)]))
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = self.resblocks(out)
        out = F.max_pool2d(out, 2)
        out = out.view(-1, 8 * 8 * self.n_chans1)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out
        

In [51]:
learning_rate = 3e-3
model = ResNetDeep()
model = model.to(device=device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
n_epochs = 300

training_loop(n_epochs, optimizer, model, loss_fn, train_loader, val_loader)

2022-03-28 15:35:28.945839 Epoch: 1, Training Loss: 1.615199089050293, Validation Loss 1.5518635511398315, Accuracy 0.3366
2022-03-28 15:38:57.893864 Epoch: 20, Training Loss: 0.9375320672988892, Validation Loss 0.8421367406845093, Accuracy 0.6235
2022-03-28 15:42:37.121219 Epoch: 40, Training Loss: 0.7135523557662964, Validation Loss 0.8400793671607971, Accuracy 0.6313
2022-03-28 15:46:16.574269 Epoch: 60, Training Loss: 0.9295684695243835, Validation Loss 1.0677989721298218, Accuracy 0.6021
2022-03-28 15:49:57.666728 Epoch: 80, Training Loss: 0.49588432908058167, Validation Loss 0.5357984900474548, Accuracy 0.668
2022-03-28 15:53:39.492817 Epoch: 100, Training Loss: 0.24813368916511536, Validation Loss 0.6217750906944275, Accuracy 0.6528
2022-03-28 15:57:23.281091 Epoch: 120, Training Loss: 0.3643956780433655, Validation Loss 0.8442394137382507, Accuracy 0.5835
2022-03-28 16:01:07.390141 Epoch: 140, Training Loss: 0.27094414830207825, Validation Loss 1.0709232091903687, Accuracy 0.62

b. Develop three additional trainings and evaluations for your ResNet-10 to assess the impacts of regularization to your 
ResNet-10.

* Weight Decay with lambda of 0.001
* Dropout with p=0.3
* Batch Normalization
Report and compare your training time, training loss, and evaluation accuracy after 300 epochs across these three different trainings. Analyze your results in your report and compare them against problem 1.a on training time, achieved accuracy

In [53]:
# 1. Weight Decay with lambda of 0.001
learning_rate = 3e-3
model = ResNetDeep()
model = model.to(device=device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.001) # Using optimizer param for weight decay
loss_fn = nn.CrossEntropyLoss()
n_epochs = 300

training_loop(n_epochs, optimizer, model, loss_fn, train_loader, val_loader)

2022-03-28 16:31:35.808076 Epoch: 1, Training Loss: 2.0224757194519043, Validation Loss 1.8004446029663086, Accuracy 0.3647
2022-03-28 16:35:05.663879 Epoch: 20, Training Loss: 0.8954801559448242, Validation Loss 1.1519253253936768, Accuracy 0.6329
2022-03-28 16:38:46.288436 Epoch: 40, Training Loss: 0.45526495575904846, Validation Loss 1.5089259147644043, Accuracy 0.6546
2022-03-28 16:42:27.231768 Epoch: 60, Training Loss: 0.7342430353164673, Validation Loss 2.1878068447113037, Accuracy 0.6545
2022-03-28 16:46:11.121129 Epoch: 80, Training Loss: 0.40167880058288574, Validation Loss 2.6666769981384277, Accuracy 0.6238
2022-03-28 16:49:58.183716 Epoch: 100, Training Loss: 0.5262927412986755, Validation Loss 2.8048529624938965, Accuracy 0.6004
2022-03-28 16:55:17.400482 Epoch: 120, Training Loss: 0.2502633333206177, Validation Loss 2.8714916706085205, Accuracy 0.6299
2022-03-28 17:01:46.102947 Epoch: 140, Training Loss: 0.26064467430114746, Validation Loss 5.066389083862305, Accuracy 0.5

In [56]:
# 2. Dropout with p=0.3
class ResBlock(nn.Module):
    def __init__(self, n_chans):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv2d(n_chans, n_chans, kernel_size=3, padding=1, bias=True)
        self.conv1_dropout = nn.Dropout2d(p=0.3)
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.conv1_dropout(out)
        out = torch.relu(out)
        return out + x
    
class ResNetDeepDropout(nn.Module):
    def __init__(self, n_chans1=32, n_blocks=10):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1, bias=True)  
        self.conv1_dropout = nn.Dropout2d(p=0.3)
        self.resblocks = nn.Sequential(
            *(n_blocks * [ResBlock(n_chans = n_chans1)]))
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = self.conv1_dropout(out)
        out = self.resblocks(out)
        out = F.max_pool2d(out, 2)
        out = out.view(-1, 8 * 8 * self.n_chans1)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [57]:
learning_rate = 3e-3
model = ResNetDeepDropout()
model = model.to(device=device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
n_epochs = 300

training_loop(n_epochs, optimizer, model, loss_fn, train_loader, val_loader)

2022-03-28 17:51:26.564704 Epoch: 1, Training Loss: 1.9746595621109009, Validation Loss 1.7630641460418701, Accuracy 0.2834
2022-03-28 17:57:02.362717 Epoch: 20, Training Loss: 1.1441643238067627, Validation Loss 1.106041431427002, Accuracy 0.5453
2022-03-28 18:01:44.017708 Epoch: 40, Training Loss: 0.9868510961532593, Validation Loss 1.0816453695297241, Accuracy 0.6042
2022-03-28 18:06:39.639702 Epoch: 60, Training Loss: 0.9195165634155273, Validation Loss 1.195173978805542, Accuracy 0.6206
2022-03-28 18:12:03.232226 Epoch: 80, Training Loss: 0.82476806640625, Validation Loss 1.0806975364685059, Accuracy 0.614
2022-03-28 18:16:52.665402 Epoch: 100, Training Loss: 0.7496321201324463, Validation Loss 1.0769792795181274, Accuracy 0.6323
2022-03-28 18:21:34.976960 Epoch: 120, Training Loss: 0.6145712733268738, Validation Loss 1.0937981605529785, Accuracy 0.6371
2022-03-28 18:26:17.566755 Epoch: 140, Training Loss: 0.6057456731796265, Validation Loss 1.4627439975738525, Accuracy 0.6314
202

In [59]:
# 3. Batch Normalization Report
class ResBlock(nn.Module):
    def __init__(self, n_chans):
        super(ResBlock, self).__init__()
        self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3,
                             padding=1, bias=False)
        self.batch_norm = nn.BatchNorm2d(num_features=n_chans)
        torch.nn.init.kaiming_normal_(self.conv.weight,
                             nonlinearity='relu')
        torch.nn.init.constant_(self.batch_norm.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm.bias)
        
    def forward(self, x):
        out = self.conv(x)
        out = self.batch_norm(out)
        out = torch.relu(out)
        return out + x
    
class ResNetDeepBatchNorm(nn.Module):
    def __init__(self, n_chans1=32, n_blocks=10):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1, bias=False)  # No bias needed as the Batch Norm handles it
        self.resblocks = nn.Sequential(
            *(n_blocks * [ResBlock(n_chans = n_chans1)]))
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = self.resblocks(out)
        out = F.max_pool2d(out, 2)
        out = out.view(-1, 8 * 8 * self.n_chans1)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out


In [60]:
learning_rate = 3e-3
model = ResNetDeepBatchNorm()
model = model.to(device=device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.001)
loss_fn = nn.CrossEntropyLoss()
n_epochs = 300

training_loop(n_epochs, optimizer, model, loss_fn, train_loader, val_loader)

2022-03-28 19:03:48.799472 Epoch: 1, Training Loss: 1.8426212072372437, Validation Loss 1.572396993637085, Accuracy 0.2984
2022-03-28 19:07:28.082877 Epoch: 20, Training Loss: 1.0498212575912476, Validation Loss 1.106888771057129, Accuracy 0.6114
2022-03-28 19:11:19.133890 Epoch: 40, Training Loss: 0.6952133178710938, Validation Loss 1.1077414751052856, Accuracy 0.6271
2022-03-28 19:15:10.046787 Epoch: 60, Training Loss: 0.8130942583084106, Validation Loss 1.0169785022735596, Accuracy 0.6627
2022-03-28 19:19:04.002297 Epoch: 80, Training Loss: 0.5004958510398865, Validation Loss 0.8437848091125488, Accuracy 0.6776
2022-03-28 19:22:54.398751 Epoch: 100, Training Loss: 0.42693597078323364, Validation Loss 0.6154416799545288, Accuracy 0.6724
2022-03-28 19:26:43.241867 Epoch: 120, Training Loss: 0.3638749420642853, Validation Loss 0.6699421405792236, Accuracy 0.657
2022-03-28 19:30:29.536296 Epoch: 140, Training Loss: 0.3209161162376404, Validation Loss 0.7202544212341309, Accuracy 0.6622


# Model Sizes

In [99]:
from ptflops import get_model_complexity_info

# Regular CNN Complexity
macs, params = get_model_complexity_info(Net(), (3, 32, 32), as_strings=True,
                                                   print_per_layer_stat=True, verbose=True)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

Net(
  0.039 M, 100.000% Params, 0.002 GMac, 100.000% MACs, 
  (conv1): Conv2d(0.001 M, 2.318% Params, 0.001 GMac, 42.987% MACs, 3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(0.005 M, 11.964% Params, 0.001 GMac, 55.461% MACs, 32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(0.033 M, 84.864% Params, 0.0 GMac, 1.537% MACs, in_features=1024, out_features=32, bias=True)
  (fc2): Linear(0.0 M, 0.854% Params, 0.0 GMac, 0.015% MACs, in_features=32, out_features=10, bias=True)
)
Computational complexity:       0.0 GMac
Number of parameters:           38.65 k 


In [100]:
# Extended CNN Complexity    
macs, params = get_model_complexity_info(ExtendedNet(), (3, 32, 32), as_strings=True,
                                                   print_per_layer_stat=True, verbose=True)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

ExtendedNet(
  0.011 M, 100.000% Params, 0.002 GMac, 100.000% MACs, 
  (conv1): Conv2d(0.001 M, 8.045% Params, 0.001 GMac, 42.088% MACs, 3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(0.005 M, 41.516% Params, 0.001 GMac, 54.302% MACs, 32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(0.001 M, 10.415% Params, 0.0 GMac, 3.406% MACs, 16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(0.004 M, 37.062% Params, 0.0 GMac, 0.189% MACs, in_features=128, out_features=32, bias=True)
  (fc2): Linear(0.0 M, 2.963% Params, 0.0 GMac, 0.015% MACs, in_features=32, out_features=10, bias=True)
)
Computational complexity:       0.0 GMac
Number of parameters:           11.14 k 


In [98]:
# ResNet based CNN Complexity    
macs, params = get_model_complexity_info(ResNetDeep(), (3, 32, 32), as_strings=True,
                                                   print_per_layer_stat=True, verbose=True)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

ResNetDeep(
  0.076 M, 100.000% Params, 0.025 GMac, 100.000% MACs, 
  (conv1): Conv2d(0.001 M, 1.136% Params, 0.001 GMac, 3.581% MACs, 3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (resblocks): Sequential(
    0.009 M, 12.204% Params, 0.024 GMac, 96.152% MACs, 
    (0): ResBlock(
      0.009 M, 12.204% Params, 0.024 GMac, 96.152% MACs, 
      (conv): Conv2d(0.009 M, 12.120% Params, 0.024 GMac, 95.489% MACs, 32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm): BatchNorm2d(0.0 M, 0.084% Params, 0.0 GMac, 0.663% MACs, 32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResBlock(
      0.009 M, 12.204% Params, 0.024 GMac, 96.152% MACs, 
      (conv): Conv2d(0.009 M, 12.120% Params, 0.024 GMac, 95.489% MACs, 32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm): BatchNorm2d(0.0 M, 0.084% Params, 0.0 GMac, 0.663% MACs, 32, eps=1e-05, momentum=0.1, affine=True, tr

In [97]:
# ResNet based CNN Complexity with weight decay regularization   
macs, params = get_model_complexity_info(ResNetDeep(), (3, 32, 32), as_strings=True,
                                                   print_per_layer_stat=True, verbose=True)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

ResNetDeep(
  0.076 M, 100.000% Params, 0.025 GMac, 100.000% MACs, 
  (conv1): Conv2d(0.001 M, 1.136% Params, 0.001 GMac, 3.581% MACs, 3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (resblocks): Sequential(
    0.009 M, 12.204% Params, 0.024 GMac, 96.152% MACs, 
    (0): ResBlock(
      0.009 M, 12.204% Params, 0.024 GMac, 96.152% MACs, 
      (conv): Conv2d(0.009 M, 12.120% Params, 0.024 GMac, 95.489% MACs, 32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm): BatchNorm2d(0.0 M, 0.084% Params, 0.0 GMac, 0.663% MACs, 32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResBlock(
      0.009 M, 12.204% Params, 0.024 GMac, 96.152% MACs, 
      (conv): Conv2d(0.009 M, 12.120% Params, 0.024 GMac, 95.489% MACs, 32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm): BatchNorm2d(0.0 M, 0.084% Params, 0.0 GMac, 0.663% MACs, 32, eps=1e-05, momentum=0.1, affine=True, tr

In [95]:
# ResNet based CNN Complexity with dropout regularization  
macs, params = get_model_complexity_info(ResNetDeepDropout(), (3, 32, 32), as_strings=True,
                                                   print_per_layer_stat=True, verbose=True)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

ResNetDeepDropout(
  0.076 M, 100.000% Params, 0.025 GMac, 100.000% MACs, 
  (conv1): Conv2d(0.001 M, 1.178% Params, 0.001 GMac, 3.709% MACs, 3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_dropout): Dropout2d(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, p=0.3, inplace=False)
  (resblocks): Sequential(
    0.009 M, 12.199% Params, 0.024 GMac, 96.025% MACs, 
    (0): ResBlock(
      0.009 M, 12.199% Params, 0.024 GMac, 96.025% MACs, 
      (conv): Conv2d(0.009 M, 12.115% Params, 0.024 GMac, 95.363% MACs, 32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm): BatchNorm2d(0.0 M, 0.084% Params, 0.0 GMac, 0.662% MACs, 32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResBlock(
      0.009 M, 12.199% Params, 0.024 GMac, 96.025% MACs, 
      (conv): Conv2d(0.009 M, 12.115% Params, 0.024 GMac, 95.363% MACs, 32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm): BatchNorm2

In [94]:
# ResNet based CNN Complexity with batch norm regularization   
macs, params = get_model_complexity_info(ResNetDeepBatchNorm(), (3, 32, 32), as_strings=True,
                                                   print_per_layer_stat=True, verbose=True)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

ResNetDeepBatchNorm(
  0.076 M, 100.000% Params, 0.025 GMac, 100.000% MACs, 
  (conv1): Conv2d(0.001 M, 1.136% Params, 0.001 GMac, 3.581% MACs, 3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (resblocks): Sequential(
    0.009 M, 12.204% Params, 0.024 GMac, 96.152% MACs, 
    (0): ResBlock(
      0.009 M, 12.204% Params, 0.024 GMac, 96.152% MACs, 
      (conv): Conv2d(0.009 M, 12.120% Params, 0.024 GMac, 95.489% MACs, 32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm): BatchNorm2d(0.0 M, 0.084% Params, 0.0 GMac, 0.663% MACs, 32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResBlock(
      0.009 M, 12.204% Params, 0.024 GMac, 96.152% MACs, 
      (conv): Conv2d(0.009 M, 12.120% Params, 0.024 GMac, 95.489% MACs, 32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm): BatchNorm2d(0.0 M, 0.084% Params, 0.0 GMac, 0.663% MACs, 32, eps=1e-05, momentum=0.1, affine